In [1]:
from functools import *
from scipy import *
from math import *
import sys
import random

In [2]:
# DE configuration
DIMENSION = 2
LOWER_BOUND = -500
UPPER_BOUND = 500
CYCLES = 5000
POP_SIZE = 50
CR, F = (0.9, 0.8)

# evaluation function
def sphere(x):
    return reduce(lambda acc,e: acc + e * e, x, 0)

def griewank(sol):
    (s,p,i) = reduce(lambda acc,e: (acc[0] + e * e, acc[1] * cos(e / sqrt(acc[2])), acc[2] + 1), sol, (0,1,1))
    return s / 4000

def schwefel(sol):
    return reduce(lambda acc,e: acc - e * sin(sqrt(abs(e))), sol, 418.9829 * DIMENSION)

def eval(x):
    return schwefel(x)

In [3]:
# population initialisation function
def init_population(dim, inf, sup, n):
    population = []
    for n in range(n):
        pos = [random.uniform(inf, sup) for i in range(dim)]
        fit = eval(pos)
        population.append({'pos': pos, 'fit': fit})
    return population

def get_best_agent(population):
    best_agent = population[0]
    for agent in population[1:]:
        if agent['fit'] < best_agent['fit']:
            best_agent = agent
    return dict(best_agent)

def update_agent(population, agent):
    new_agent = dict(agent)
    dim = len(new_agent["pos"])
    
    temp_population = list(filter(lambda a: a is not agent, population))
    # random selection of 3 individuals, distincts of themselves and of the currently concerned agent
    a,b,c = random.choices(temp_population, k=3)
    # ensured component heritance (index)
    index = random.choice(list(range(dim))) 
    
    new_pos = [0] * dim
    for i in range(dim):
        ri = random.uniform(0,1)
        if ri < CR or i == index: # second condition to ensure that at least one component of the vector is herited
            new_pos[i] = bornage(
                a['pos'][i] + F * (b['pos'][i] - c['pos'][i]), LOWER_BOUND, UPPER_BOUND)
        else:
            new_pos[i] = new_agent['pos'][i]
    new_eval = eval(new_pos)
    if new_eval <= new_agent['fit']:
        new_agent['pos'] = new_pos[:]
        new_agent['fit'] = new_eval
    
    return new_agent

def bornage(val, inf, sup):
    return min (sup, max (inf, val))

def display_result(best_agent):
    print("point = {}".format(best_agent['pos']))
    print("eval = {}".format(best_agent['fit']))

In [4]:
population = init_population(DIMENSION, LOWER_BOUND, UPPER_BOUND, POP_SIZE)
best_global = get_best_agent(population)
best_local = best_global

for i in range(CYCLES):
    population = [update_agent(population, agent) for agent in population]
    best_local = get_best_agent(population)
    if (best_local["fit"] < best_global["fit"]):
        best_global = best_local
display_result(best_global)

point = [420.9687465083046, 420.968746689436]
eval = 2.5455132345086895e-05
